### **Obtener el historico de los precios de las 10 criptomonedas**

Se utiliza la API de CoinGecko para obtener datos históricos de precios de un conjunto de criptomonedas. Luego, se procesan estos datos para crear un DataFrame consolidado que contiene los precios de las criptomonedas a lo largo de un período de tiempo especificado. El código también maneja casos en los que no se encuentran datos para alguna de las criptomonedas en el rango de tiempo especificado.

In [4]:
import requests
import pandas as pd

# Lista de criptomonedas sugeridas junto con sus nombres
cryptocurrencies = [
    ("bitcoin", "Bitcoin"), ("ethereum", "Ethereum"), ("binancecoin", "Binance Coin"),
    ("aave", "Aave"), ("solana", "Solana"), ("polkadot", "Polkadot"),
    ("chainlink", "Chainlink"), ("avalanche-2", "Avalanche"), ("algorand", "Algorand"),
    ("matic-network", "Polygon")]

# Parámetros de la URL
base_url = "https://api.coingecko.com/api/v3/coins"
vs_currency = "usd"
from_timestamp = 1230786000  # Timestamp de inicio
to_timestamp = 1691643599    # Timestamp de fin
precision = 2

# Lista para almacenar los DataFrames de cada criptomoneda
data_frames = []

# Iterar sobre las criptomonedas y obtener los datos
for cryptocurrency, name in cryptocurrencies:
    url = f"{base_url}/{cryptocurrency}/market_chart/range"
    params = {
        "vs_currency": vs_currency,
        "from": from_timestamp,
        "to": to_timestamp,
        "precision": precision
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if "prices" in data:
        df = pd.DataFrame(data["prices"], columns=["Fecha", name])
        df["Fecha"] = pd.to_datetime(df["Fecha"], unit="ms")
        df[name] = df[name].astype(float)
        
        data_frames.append(df)

# Si hay al menos un DataFrame en data_frames, procedemos a combinarlos y ordenarlos
if data_frames:
    final_df = data_frames[0]
    for df in data_frames[1:]:
        final_df = pd.merge(final_df, df, on="Fecha", how="outer")
    
    # Ordenar el DataFrame por la columna "Fecha"
    final_df = final_df.sort_values(by="Fecha")
    
    # Reemplazar NaN con 0
    final_df = final_df.fillna(0)
    
    # Suponiendo que 'Fecha' es una columna en formato de cadena de texto
    final_df['Fecha'] = pd.to_datetime(final_df['Fecha']).dt.date
    
    # Imprimir el DataFrame final
    final_df
else:
    print("No se encontraron datos para ninguna criptomoneda en el rango de tiempo especificado.")

In [5]:
final_df

,Fecha,Bitcoin,Ethereum,Binance Coin,Aave,Solana,Polkadot,Chainlink,Avalanche,Algorand,Polygon
0,2013-04-28,135.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2013-04-29,141.96,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2013-04-30,135.30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2013-05-01,117.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2013-05-02,103.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
3750,2023-08-06,29046.80,1835.07,242.88,64.37,22.63,4.97,7.22,12.43,0.11,0.67
3751,2023-08-07,29044.20,1826.71,242.57,65.76,23.16,4.99,7.09,12.56,0.11,0.67
3752,2023-08-08,29178.15,1826.93,241.74,64.85,23.05,4.96,7.28,12.45,0.11,0.66
3753,2023-08-09,29779.56,1856.84,245.18,66.29,24.20,5.06,7.45,12.69,0.11,0.69


In [6]:
# Guardar el DataFrame en un archivo CSV
final_df.to_csv('Dataset//data_price.csv', index=False)

### **Obtener caracteristicas de las criptomonedas**


Se utiliza la API de CoinGecko para obtener información sobre un conjunto de criptomonedas específicas. Los datos recuperados se procesan y se crea un DataFrame que muestra características relevantes de estas criptomonedas, como el nombre, el precio actual, la capitalización de mercado, el rango de capitalización, el suministro total y el dominio del mercado. Además, el código formatea los valores de las columnas pertinentes en formato de moneda USD y calcula la capitalización de mercado total, así como el dominio del mercado para cada criptomoneda. 

In [1]:
import requests
import pandas as pd
import locale

# URL de la API para obtener la capitalización total del mercado global
url_global = "https://api.coingecko.com/api/v3/global"
response_global = requests.get(url_global)
data_global = response_global.json()

total_market_cap = data_global["data"]["total_market_cap"]["usd"]  # Obtener la capitalización total en USD

# URL de la API para obtener datos de criptomonedas
url = "https://api.coingecko.com/api/v3/coins/markets"

# Lista de criptomonedas junto con sus identificadores
cryptos = [
    ("bitcoin", "Bitcoin"),
    ("ethereum", "Ethereum"),
    ("binancecoin", "Binance Coin"),
    ("aave", "Aave"),
    ("solana", "Solana"),
    ("polkadot", "Polkadot"),
    ("chainlink", "Chainlink"),
    ("avalanche-2", "Avalanche"),
    ("algorand", "Algorand"),
    ("matic-network", "Polygon"),
]

# Parámetros de la solicitud
params = {
    "vs_currency": "usd",
    "ids": ",".join(crypto[0] for crypto in cryptos),
    "order": "market_cap_desc",
    "per_page": 11,
    "page": 1,
    "sparkline": "false",
    "price_change_percentage": "",
    "locale": "en",
    "precision": 2
}

# Realizar la solicitud HTTP
response = requests.get(url, params=params)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    data = response.json()  
    # Crear un DataFrame a partir de los datos
    caracteristicas_cryptos = pd.DataFrame(data)
    # Extraer las columnas necesarias
    columnas_deseadas = ["id", "symbol", "name", "current_price", "market_cap", "market_cap_rank", "total_supply"]
    caracteristicas_cryptos = caracteristicas_cryptos[columnas_deseadas]
    
    # Convertir valores de market_cap a números
    caracteristicas_cryptos["market_cap"] = pd.to_numeric(caracteristicas_cryptos["market_cap"], errors='coerce')
    
    # Calcular el Dominio del Mercado para cada criptomoneda
    caracteristicas_cryptos["market_dominance"] = (caracteristicas_cryptos["market_cap"] / total_market_cap) * 100
    caracteristicas_cryptos["market_dominance"] = caracteristicas_cryptos["market_dominance"].round(2)  # Redondear a 2 decimales
    
    # Resto del código para formatear y mostrar la información
    # Establecer el locale para formatear como USD
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    
    # Formatear columnas en formato USD
    columnas_formateadas = ["current_price", "market_cap","total_supply"]
    caracteristicas_cryptos[columnas_formateadas] = caracteristicas_cryptos[columnas_formateadas].applymap(lambda x: locale.currency(float(str(x).replace(',', '')), grouping=True))
    
    caracteristicas_cryptos
    
    print("Capitalización de Mercado Total: ", locale.currency(total_market_cap, grouping=True))
else:
    print("Error en la solicitud. Código de estado:", response.status_code)


Capitalización de Mercado Total:  $1,218,002,383,162.87


In [4]:
caracteristicas_cryptos.rename(columns={
    "id": "id",
    "symbol": "símbolo",
    "name": "nombre",
    "current_price": "precio_actual",
    "market_cap": "capitalización_mercado",
    "market_cap_rank": "rango_capitalización_mercado",
    "total_supply": "suministro_total",
    "market_dominance": "dominio_mercado"
}, inplace=True)

In [5]:
caracteristicas_cryptos

,id,símbolo,nombre,precio_actual,capitalización_mercado,rango_capitalización_mercado,suministro_total,dominio_mercado
0,bitcoin,btc,Bitcoin,"$29,361.39","$571,267,754,769.00",1,"$21,000,000.00",46.90
1,ethereum,eth,Ethereum,"$1,843.49","$221,434,517,577.00",2,"$120,144,363.74",18.18
2,binancecoin,bnb,BNB,$240.15,"$36,958,957,222.00",4,"$153,856,150.00",3.03
3,solana,sol,Solana,$24.86,"$10,095,982,592.00",10,"$554,933,656.84",0.83
4,matic-network,matic,Polygon,$0.68,"$6,331,103,280.00",12,"$10,000,000,000.00",0.52
5,polkadot,dot,Polkadot,$4.98,"$6,300,160,350.00",13,"$1,345,287,496.75",0.52
6,avalanche-2,avax,Avalanche,$12.33,"$4,240,227,108.00",20,"$430,547,431.69",0.35
7,chainlink,link,Chainlink,$7.44,"$4,005,344,544.00",21,"$1,000,000,000.00",0.33
8,aave,aave,Aave,$65.60,"$951,791,401.00",46,"$16,000,000.00",0.08
9,algorand,algo,Algorand,$0.12,"$900,673,420.00",47,"$7,812,954,146.58",0.07


In [6]:
# Guardar el DataFrame en un archivo CSV
caracteristicas_cryptos.to_csv('Dataset//caracteristicas_cryptos.csv', index=False)